In [1]:
import sparknlp
spark = sparknlp.start() 
# sparknlp.start(gpu=True) >> for training on GPUfrom sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd
print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

23/02/16 20:10:14 WARN Utils: Your hostname, Vader resolves to a loopback address: 127.0.1.1; using 192.168.1.94 instead (on interface enp5s0f0)
23/02/16 20:10:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/sjhjrok/anaconda3/envs/NLP/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sjhjrok/.ivy2/cache
The jars for the packages stored in: /home/sjhjrok/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-71219f90-a308-4052-a105-c56e806238be;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.2.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#l

23/02/16 20:10:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark NLP version 4.2.2
Apache Spark version: 3.3.1


In [2]:
#! wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_train.csv

In [3]:
#! wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_test.csv

In [4]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("text_train.csv")
trainDataset.show(10, truncate=50)

+--------------------------------+--------+
|                     description|category|
+--------------------------------+--------+
|           i want to make coffee|      DA|
|                      where am i|      LR|
|              what can i do here|      AR|
|                    prepare meal|      DA|
|                use refrigerator|      DA|
|                         use fan|      DA|
|                        use oven|      DA|
|                       use stove|      DA|
|     i would like to wash sheets|      DA|
|i would like to watch television|      DA|
+--------------------------------+--------+
only showing top 10 rows



In [5]:
from pyspark.sql.functions import col
trainDataset.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
|category|count|
+--------+-----+
|      DA|   24|
|      SN|   20|
|      LR|   14|
|      VC|   13|
|      AR|   13|
|     NAV|   13|
|      SD|   12|
|      CC|   11|
|      NA|    1|
+--------+-----+



In [6]:
testDataset = spark.read \
      .option("header", True) \
      .csv("text_test.csv")
testDataset.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
|category|count|
+--------+-----+
|      DA|   24|
|      SN|   20|
|      LR|   14|
|      VC|   13|
|      AR|   13|
|     NAV|   13|
|      SD|   12|
|      CC|   11|
|      NA|    1|
+--------+-----+



In [7]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")
    
# we can also use sentence detector here 
# if we want to train on and get predictions for each sentence# downloading pretrained embeddings
use = UniversalSentenceEncoder.pretrained()\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")
# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("category")\
    .setMaxEpochs(50)\
    .setLr(0.003)\
    .setBatchSize(32)\
    .setRandomSeed(0)\
    .setVerbose(1)\
    .setValidationSplit(0.2)\
    .setEvaluationLogExtended(True) \
    .setEnableOutputLogs(True)\
    .setOutputLogsPath('classifer_logs')\

use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

use_pipelineModel = use_clf_pipeline.fit(trainDataset)


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ]tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
Download done! Loading the resource.
[ \ ]

2023-02-16 20:10:32.568114: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[ | ]

2023-02-16 20:10:34.749380: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 60236800 exceeds 10% of free system memory.
2023-02-16 20:10:34.854650: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 60236800 exceeds 10% of free system memory.
2023-02-16 20:10:34.903243: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 60236800 exceeds 10% of free system memory.
2023-02-16 20:10:34.952851: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 60236800 exceeds 10% of free system memory.
2023-02-16 20:10:34.991312: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 60236800 exceeds 10% of free system memory.


[ / ]

[OK!]


2023-02-16 20:10:42.540571: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/3e06455cb488_classifier_dl17815385120324979816
2023-02-16 20:10:42.569995: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2023-02-16 20:10:42.570116: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/3e06455cb488_classifier_dl17815385120324979816
2023-02-16 20:10:42.839194: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-02-16 20:10:43.253156: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/3e06455cb488_classifier_dl17815385120324979816
2023-02-16 20:10:43.355668: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 815107 microsec

Training started - epochs: 50 - learning_rate: 0.003 - batch_size: 32 - training_examples: 97 - classes: 9
Epoch 1/50 - 0.21s - loss: 8.46114 - acc: 0.5729167 - batches: 4
Quality on validation dataset (20.0%), validation examples = 24
time to finish evaluation: 0.16s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           6	 17	 0	 0.26086956	 1.0	 0.41379312
NAV          0	 0	 2	 0.0	 0.0	 0.0
SD           0	 0	 2	 0.0	 0.0	 0.0
VC           0	 0	 3	 0.0	 0.0	 0.0
CC           0	 0	 2	 0.0	 0.0	 0.0
AR           0	 0	 3	 0.0	 0.0	 0.0
SN           1	 0	 3	 1.0	 0.25	 0.4
LR           0	 0	 2	 0.0	 0.0	 0.0
tp: 7 fp: 17 fn: 17 labels: 8
Macro-average	 prec: 0.15760869, rec: 0.15625, f1: 0.15692641
Micro-average	 prec: 0.29166666, recall: 0.29166666, f1: 0.29166666
Epoch 2/50 - 0.01s - loss: 7.137374 - acc: 0.6666667 - batches: 4
Quality on validation dataset (20.0%), validation examples = 24
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           5	 11	 1	 0.3

Epoch 20/50 - 0.01s - loss: 6.39395 - acc: 1.0 - batches: 4
Quality on validation dataset (20.0%), validation examples = 24
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           4	 0	 2	 1.0	 0.6666667	 0.8
NAV          1	 0	 1	 1.0	 0.5	 0.6666667
SD           0	 0	 2	 0.0	 0.0	 0.0
VC           2	 5	 1	 0.2857143	 0.6666667	 0.4
CC           2	 0	 0	 1.0	 1.0	 1.0
AR           0	 0	 3	 0.0	 0.0	 0.0
SN           4	 1	 0	 0.8	 1.0	 0.88888896
LR           2	 3	 0	 0.4	 1.0	 0.5714286
tp: 15 fp: 9 fn: 9 labels: 8
Macro-average	 prec: 0.5607143, rec: 0.6041667, f1: 0.58163005
Micro-average	 prec: 0.625, recall: 0.625, f1: 0.625
Epoch 21/50 - 0.01s - loss: 6.3954644 - acc: 1.0 - batches: 4
Quality on validation dataset (20.0%), validation examples = 24
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           4	 0	 2	 1.0	 0.6666667	 0.8
NAV          1	 0	 1	 1.0	 0.5	 0.6666667
SD           0	 0	 2	 0.0	 0.0	 0.0
VC           2	 

Epoch 38/50 - 0.01s - loss: 6.420145 - acc: 1.0 - batches: 4
Quality on validation dataset (20.0%), validation examples = 24
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           4	 0	 2	 1.0	 0.6666667	 0.8
NAV          2	 2	 0	 0.5	 1.0	 0.6666667
SD           0	 0	 2	 0.0	 0.0	 0.0
VC           2	 2	 1	 0.5	 0.6666667	 0.57142854
CC           2	 0	 0	 1.0	 1.0	 1.0
AR           0	 1	 3	 0.0	 0.0	 0.0
SN           4	 1	 0	 0.8	 1.0	 0.88888896
LR           2	 2	 0	 0.5	 1.0	 0.6666667
tp: 16 fp: 8 fn: 8 labels: 8
Macro-average	 prec: 0.5375, rec: 0.6666667, f1: 0.5951557
Micro-average	 prec: 0.6666667, recall: 0.6666667, f1: 0.6666667
Epoch 39/50 - 0.01s - loss: 6.41949 - acc: 1.0 - batches: 4
Quality on validation dataset (20.0%), validation examples = 24
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
DA           4	 0	 2	 1.0	 0.6666667	 0.8
NAV          2	 2	 0	 0.5	 1.0	 0.6666667
SD           0	 0	 2	 0.0	 0.0	 0.0
VC      

In [8]:
use_pipelineModel.stages[2].write().overwrite().save('Text_Classification')

In [9]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")
    
# we can also use sentence detector here 
# if we want to train on and get predictions for each sentence# downloading pretrained embeddings
use = UniversalSentenceEncoder.pretrained()\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")
# the classes/labels/categories are in category column
loaded_ner_model = ClassifierDLModel.load("Text_Classification")\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")

use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        loaded_ner_model
    ])

#use_pipelineModel = use_clf_pipeline.fit(trainDataset)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [10]:
from sparknlp.base import LightPipeline
#clf_pipelineModel = use_clf_pipeline.fit(trainDataset)

text = "navigate to kitchen"
prediction_data = spark.createDataFrame([[text]]).toDF("text")
prediction_model = use_clf_pipeline.fit(prediction_data)
light = LightPipeline(prediction_model)
results = light.annotate(text)
print(results)
print(results['class'])

{'document': ['navigate to kitchen'], 'sentence_embeddings': ['navigate to kitchen'], 'class': ['NAV']}
['NAV']


In [ ]:
text = input("Enter Testing Text\n")
while(text != "exit"):
    results = light.annotate(text)
    print(results['class'])
    text = input("Enter New Text\n")

Enter Testing Text
open camera
['CC']
Enter New Text
start camera
['CC']
Enter New Text
rewind video
['VC']
